<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/Mobilenet_Base_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:



"""Mobilenet Base Class."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import collections
import contextlib
import copy
import os

import tensorflow as tf


slim = tf.contrib.slim


@slim.add_arg_scope
def apply_activation(x, name=None, activation_fn=None):
  return activation_fn(x, name=name) if activation_fn else x


def _fixed_padding(inputs, kernel_size, rate=1):
'''
Дополняет ввод по пространственным измерениям независимо от размера ввода.

Дополняет ввод таким образом, чтобы, если он использовался в свертке с 'VALID' padding, выход будет иметь такие же размеры, 
как если бы использовался незаполненный вход в свертке с 'SAME' padding.

Args:
   inputs: A tensor of size [batch, height_in, width_in, channels].
   kernel_size: The kernel to be used in the conv2d or max_pool2d operation.
   rate: целое число, скорость для свертывания
Returns:
   output: A tensor of size [batch, height_out, width_out, channels] with the input, либо нетронутым (if kernel_size == 1) либо с дополнением (if kernel_size > 1).
'''
  kernel_size_effective = [kernel_size[0] + (kernel_size[0] - 1) * (rate - 1),
                           kernel_size[0] + (kernel_size[0] - 1) * (rate - 1)]
  pad_total = [kernel_size_effective[0] - 1, kernel_size_effective[1] - 1]
  pad_beg = [pad_total[0] // 2, pad_total[1] // 2]
  pad_end = [pad_total[0] - pad_beg[0], pad_total[1] - pad_beg[1]]
  padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg[0], pad_end[0]],
                                  [pad_beg[1], pad_end[1]], [0, 0]])
  return padded_inputs


# название - сделать делимым
def _make_divisible(v, divisor, min_value=None):
  if min_value is None:
    min_value = divisor
  new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
  #Убедитесь, что округление вниз не опускается более чем на 10%.
  if new_v < 0.9 * v:
    new_v += divisor
  return new_v




@contextlib.contextmanager
def _set_arg_scope_defaults(defaults):
'''
Устанавливает значения по умолчанию для arg scope для всех элементов, присутствующих в значениях по умолчанию.

Args:
  defaults: dictionary/list of pairs, containing a mapping from function to a dictionary of default args.
  
Yields:
  context manager в котором установлены все значения по умолчанию.
'''
  if hasattr(defaults, 'items'):
    items = list(defaults.items())
  else:
    items = defaults
  if not items:
    yield
  else:
    func, default_arg = items[0]
    with slim.arg_scope(func, **default_arg):
      with _set_arg_scope_defaults(items[1:]):
        yield


@slim.add_arg_scope
def depth_multiplier(output_params,
                     multiplier,
                     divisible_by=8,
                     min_depth=8,
                     **unused_kwargs):
  if 'num_outputs' not in output_params:
    return
  d = output_params['num_outputs']
  output_params['num_outputs'] = _make_divisible(d * multiplier, divisible_by,
                                                 min_depth)


_Op = collections.namedtuple('Op', ['op', 'params', 'multiplier_func'])


def op(opfunc, **params):
  multiplier = params.pop('multiplier_transorm', depth_multiplier)
  return _Op(opfunc, params=params, multiplier_func=multiplier)


class NoOpScope(object):
  """No-op context manager."""

  def __enter__(self):
    return None

  def __exit__(self, exc_type, exc_value, traceback):
    return False





def safe_arg_scope(funcs, **kwargs):
"""Returns `slim.arg_scope` with all None arguments removed.

  Arguments:
    funcs: Functions to pass to `arg_scope`.
    **kwargs: Arguments для передачи в `arg_scope`.

  Returns:
    arg_scope or No-op context manager.

Примечание: может быть полезно, если значение None следует интерпретировать как «не перезаписывать» это значение параметра ".
"""
  filtered_args = {name: value for name, value in kwargs.items()
                   if value is not None}
  if filtered_args:
    return slim.arg_scope(funcs, **filtered_args)
  else:
    return NoOpScope()


@slim.add_arg_scope
def mobilenet_base(  # pylint: disable=invalid-name
    inputs,
    conv_defs,
    multiplier=1.0,
    final_endpoint=None,
    output_stride=None,
    use_explicit_padding=False,
    scope=None,
    is_training=False):
  """Mobilenet base network.

  Создает сеть от input до заданной конечной endpoint. По умолчанию сеть построена в режиме вывода. 
  Создать сеть в режиме тренировки используйте:
  
  with slim.arg_scope(mobilenet.training_scope()):
     logits, endpoints = mobilenet_base(...)
            
   Args:
     inputs: тензор формы [batch_size, height, width, channels].
     conv_defs: список op(...) слоев, определяющих архитектуру сети.
     multiplier: множитель с плавающей запятой для глубины (количество каналов)для всех операций свертки. 
        Значение должно быть больше нуля. Типичный будет использоваться для установки этого значения в (0, 1),
        чтобы уменьшить количество параметры или стоимость расчета модели.
     final_endpoint: имя последнего слоя, для досрочного завершения для сетей на основе V1: последний уровень - "layer_14", для V2: "layer_20"
     output_stride: целое число, которое указывает запрошенное отношение ввода к выходное пространственное разрешение. 
        Если не None, то мы вызываем зловещую сверткуесли необходимо, чтобы сеть не уменьшала пространственное разрешение карты активации. 
        Допустимые значения: 1 или любое четное число, кроме нуль. Типичные значения: 8 (точный полностью сверточный режим), 
        16(быстрый полностью сверточный режим) и 32 (режим классификации).

   ПРИМЕЧАНИЕ - output_stride опирается на все последующие операторы для поддержки расширенных операторов через параметр «rate».
   Это может потребовать переноса неконвенционных операторов для правильной работы.

    use_explicit_padding: Используйте заполнение 'VALID' для сверток, но вводите входные данные таким образом, чтобы выходные размеры
        были такими же, как если бы использовалось заполнение 'SAME'. 
    scope: необязательная переменная область действия. 
    is_training: как настроить batch_norm и другие операции. Примечание: большую часть времени это не нужно устанавливать напрямую. 
        Вместо этого используйте mobilenet.training_scope () для настройки обучения. Этот параметр здесь только для обратной совместимости. 
        Безопасно установить его в значение, соответствующее training_scope (is_training = ...). Также безопасно явно установить его в False,
        даже если для external training_scope установлено значение обучения. (Сеть будет построена в режиме вывода).
        Если для этого параметра задано значение None, параметр arg_scope для параметра slim_batch_norm is_training не добавляется.

  Returns:
    tensor_out: output tensor.
    end_points: набор активаций для внешнего использования, например, summaries or losses.
  Raises:
    ValueError: depth_multiplier <= 0, or the target output_stride is not
                allowed.
  """
  if multiplier <= 0:
    raise ValueError('multiplier is not greater than zero.')

  # Set conv defs defaults and overrides.
  conv_defs_defaults = conv_defs.get('defaults', {})
  conv_defs_overrides = conv_defs.get('overrides', {})
  if use_explicit_padding:
    conv_defs_overrides = copy.deepcopy(conv_defs_overrides)
    conv_defs_overrides[
        (slim.conv2d, slim.separable_conv2d)] = {'padding': 'VALID'}

  if output_stride is not None:
    if output_stride == 0 or (output_stride > 1 and output_stride % 2):
      raise ValueError('Output stride must be None, 1 or a multiple of 2.')

  # a) Set the tensorflow scope
  # b) set padding to default: note we might consider removing this
  # since it is also set by mobilenet_scope
  # c) set all defaults
  # d) set all extra overrides.
  with _scope_all(scope, default_scope='Mobilenet'), \
      safe_arg_scope([slim.batch_norm], is_training=is_training), \
      _set_arg_scope_defaults(conv_defs_defaults), \
      _set_arg_scope_defaults(conv_defs_overrides):
    #Переменная current_stride отслеживает ход вывода активаций, 
    #то есть текущий продукт свертки продвигается к текущему сетевому уровню. 
    #Это позволяет нам вызывать атральную свертку всякий раз, когда применение 
    #следующей свертки приведет к тому, что у активаций будет выходной шаг больше, чем целевой output_stride.
    current_stride = 1

    # The atrous convolution rate parameter.
    rate = 1

    net = inputs
    # Вставьте параметры по умолчанию перед базовой областью, которая включает
    # любые пользовательские переопределения, установленные в mobilenet.
    end_points = {}
    scopes = {}
    for i, opdef in enumerate(conv_defs['spec']):
      params = dict(opdef.params)
      opdef.multiplier_func(params, multiplier)
      stride = params.get('stride', 1)
      if output_stride is not None and current_stride == output_stride:
    # Если мы достигли цели output_stride, то нам нужно использовать 
    # atrous convolution с шагом = 1 и умножить скорость сердцебиения на 
    # шаг текущего юнита для использования в последующих слоях.
        layer_stride = 1
        layer_rate = rate
        rate *= stride
      else:
        layer_stride = stride
        layer_rate = 1
        current_stride *= stride
      # Update params.
      params['stride'] = layer_stride
      # Only insert rate to params if rate > 1.
      if layer_rate > 1:
        params['rate'] = layer_rate
      # Set padding
      if use_explicit_padding:
        if 'kernel_size' in params:
          net = _fixed_padding(net, params['kernel_size'], layer_rate)
        else:
          params['use_explicit_padding'] = True

      end_point = 'layer_%d' % (i + 1)
      try:
        net = opdef.op(net, **params)
      except Exception:
        print('Failed to create op %i: %r params: %r' % (i, opdef, params))
        raise
      end_points[end_point] = net
      scope = os.path.dirname(net.name)
      scopes[scope] = end_point
      if final_endpoint is not None and end_point == final_endpoint:
        break

    # Add all tensors that end with 'output' to
    # endpoints
    for t in net.graph.get_operations():
      scope = os.path.dirname(t.name)
      bn = os.path.basename(t.name)
      if scope in scopes and t.name.endswith('output'):
        end_points[scopes[scope] + '/' + bn] = t.outputs[0]
    return net, end_points


@contextlib.contextmanager
def _scope_all(scope, default_scope=None):
  with tf.variable_scope(scope, default_name=default_scope) as s,\
       tf.name_scope(s.original_name_scope):
    yield s


@slim.add_arg_scope
def mobilenet(inputs,
              num_classes=1001,
              prediction_fn=slim.softmax,
              reuse=None,
              scope='Mobilenet',
              base_only=False,
              **mobilenet_args):
  """Mobilenet model for classification, supports both V1 and V2.

Примечание: режим по умолчанию - логический вывод, используйте mobilenet.training_scope для создания обучающей сети.


  Args:
    inputs: a tensor of shape [batch_size, height, width, channels].
    num_classes: количество предсказанных классов. Если 0 или Нет, слой logits опущен, 
      и вместо него возвращаются входные объекты для слоя logits (до выпадения).
    prediction_fn: функция для получения прогнозов из логитов (по умолчанию softmax).
    reuse: следует ли повторно использовать сеть и ее переменные. Для возможности повторного 
      использования 'scope' должна быть предоставлена.
    scope: Optional variable_scope.
    base_only: если True только создаст базу сети (без пула и без логитов)..
    **mobilenet_args: passed to mobilenet_base verbatim.
      - conv_defs: list of conv defs
      - multiplier: множитель с плавающей запятой для глубины (количества каналов) для всех 
        операций свертки. Значение должно быть больше нуля. Обычно используется для установки 
        этого значения в (0, 1), чтобы уменьшить количество параметров или стоимость вычислений модели.
      - output_stride: будет гарантировать, что последний слой имеет не более общего шага. 
      Если архитектура требует большего шага, чем предусмотрено (например, output_stride = 16, 
      но архитектура имеет 5 stride = 2 оператора), она заменит output_stride с дробными 
      сверткой, используя Atrous Convolutions.

  Returns:
    logits: the pre-softmax activations, a tensor of size
      [batch_size, num_classes]
    end_points: a dictionary from components of the network to the corresponding
      activation tensor.

  Raises:
    ValueError: Input rank is invalid.
  """
  is_training = mobilenet_args.get('is_training', False)
  input_shape = inputs.get_shape().as_list()
  if len(input_shape) != 4:
    raise ValueError('Expected rank 4 input, was: %d' % len(input_shape))

  with tf.variable_scope(scope, 'Mobilenet', reuse=reuse) as scope:
    inputs = tf.identity(inputs, 'input')
    net, end_points = mobilenet_base(inputs, scope=scope, **mobilenet_args)
    if base_only:
      return net, end_points

    net = tf.identity(net, name='embedding')

    with tf.variable_scope('Logits'):
      net = global_pool(net)
      end_points['global_pool'] = net
      if not num_classes:
        return net, end_points
      net = slim.dropout(net, scope='Dropout', is_training=is_training)
      # 1 x 1 x num_classes
      # Note: legacy scope name.
      logits = slim.conv2d(
          net,
          num_classes, [1, 1],
          activation_fn=None,
          normalizer_fn=None,
          biases_initializer=tf.zeros_initializer(),
          scope='Conv2d_1c_1x1')

      logits = tf.squeeze(logits, [1, 2])

      logits = tf.identity(logits, name='output')
    end_points['Logits'] = logits
    if prediction_fn:
      end_points['Predictions'] = prediction_fn(logits, 'Predictions')
  return logits, end_points


def global_pool(input_tensor, pool_op=tf.nn.avg_pool):
  """Применяет пул avg для вывода 1x1.

   ПРИМЕЧАНИЕ. Эта функция функционально эквивалентна limit_mean, но имеет средний пул, который имеет лучшую поддержку для всего оборудования.

   Args:
     input_tensor: входной тензор
     pool_op: объединение в пул (средний пул по умолчанию)
   Возвращает:
     тензор batch_size x 1 x 1 x глубина.
  """
  shape = input_tensor.get_shape().as_list()
  if shape[1] is None or shape[2] is None:
    kernel_size = tf.convert_to_tensor(
        [1, tf.shape(input_tensor)[1],
         tf.shape(input_tensor)[2], 1])
  else:
    kernel_size = [1, shape[1], shape[2], 1]
  output = pool_op(
      input_tensor, ksize=kernel_size, strides=[1, 1, 1, 1], padding='VALID')
  # Recover output shape, for unknown shape.
  output.set_shape([None, 1, 1, None])
  return output


def training_scope(is_training=True,
                   weight_decay=0.00004,
                   stddev=0.09,
                   dropout_keep_prob=0.8,
                   bn_decay=0.997):
  """Определяет область обучения Mobilenet.

  Usage:
     with tf.contrib.slim.arg_scope(mobilenet.training_scope()):
       logits, endpoints = mobilenet_v2.mobilenet(input_tensor)

   # созданная сеть будет работоспособна с dropout/batch norm инициализирован соответствующим образом.
  
  Args:
    is_training: если установлено значение False, это гарантирует, что все настройки будут 
    установлены в режим без обучения. Это может быть полезно для кода, который повторно 
    используется для обучения / оценки, но большую часть времени training_scope со значением
    False не требуется. Если для этого параметра установлено значение None, параметры не 
    добавляются в batch_norm arg_scope.

    weight_decay: Снижение веса использовать для регуляризации модели.
    stddev: Стандартное отклонение для инициализации, если отрицательное значение использует xavier.
    dropout_keep_prob: вероятность dropout (не установлена, если равна None).
    bn_decay: затухание для скользящих средних batch norm (не устанавливается, если равно None).

  Returns:
    An argument scope to use via arg_scope.
  """
# Примечание: не вводите параметры, которые могли бы изменить модель логического вывода здесь 
# (например, использовать ли смещение), вместо этого измените conv_def.
  batch_norm_params = {
      'decay': bn_decay,
      'is_training': is_training
  }
  if stddev < 0:
    weight_intitializer = slim.initializers.xavier_initializer()
  else:
    weight_intitializer = tf.truncated_normal_initializer(stddev=stddev)

  # Установите weight_decay для весов в слоях Conv и FC.
  with slim.arg_scope(
      [slim.conv2d, slim.fully_connected, slim.separable_conv2d],
      weights_initializer=weight_intitializer,
      normalizer_fn=slim.batch_norm), \
      slim.arg_scope([mobilenet_base, mobilenet], is_training=is_training),\
      safe_arg_scope([slim.batch_norm], **batch_norm_params), \
      safe_arg_scope([slim.dropout], is_training=is_training,
                     keep_prob=dropout_keep_prob), \
      slim.arg_scope([slim.conv2d], \
                     weights_regularizer=slim.l2_regularizer(weight_decay)), \
      slim.arg_scope([slim.separable_conv2d], weights_regularizer=None) as s:
    return s